# Matrix Inversion as a QUBO problem
This notebook is an extension and update to the implementation of the paper "Floating-Point Calculations on a Quantum Annealer: Division and Matrix Inversion" by M. Rogers and R. Singleton, 2020 [1]. 

The paper describes an approach to floating-point calculations on a quantum annealer. Specifically, they introduce a binary representation of the floating point variables. This representation is discrete, as they demonstrate their technique for 4 bit and 8 bit accuracy. With this measure they explain and derive the inversion of a matrix as a QUBO problem, which is suitable to run on a DWave quantum annealer. 

We plan to expand the discrete (limited floating-point accuracy) to an integer problem solvable on the new hybrid solvers from DWave. In addition to solving integer style problems, the new hybrid solvers allow the integration of mathematical constraints. In the context of tomographic reconstruction for example non-negativity constraints are frequently used.

## Formulating the problem for the quantum annealer

The main challenge in formulating problems for quantum annealers is the description as a quadratic binary problem. Previously Rogers and Singleton have described matrix inversion as a quadratic binary problem for floating-point precision [1].   

Scalar notation of a QUBO problem: 

\begin{equation} E_{qubo}(a_i, b_{i,j}; q_i) = \sum_{i} a_i q_i + \sum_{i < j} b_{i, j} q_i q_j 
\end{equation} 


- $q_i$: Unknown binary variable (0, 1) we want to optimize for 
- $a_i$: Qubit bias/weight (real number)
- $b_{i,j}$: Coupling strength between spins $i$ and $j$ (real number)




Matrix (NxN) inversion is defined as: 

\begin{equation} M \cdot x = y \rightarrow x=M^{-1} \cdot y 
\end{equation} 

Formulate matrix inversion as a quadratic minimization problem with its minimum being the matrix inverse:

\begin{equation} H(x) = (Mx-y)^2 = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2 \sum_{ij=1}^{N} y_j M_{ji} x^i + \| y \|^2 = x^T M^T M x - x^T M^T y - y^t M x + y^T y
\end{equation} 

In matrix inversion and solving a linear system we usually solve for a vector of numbers which are typically not binary. Previously the common practice to obtain a floating point or integer representation of each component of x was to expand in powers of 2 multiplied by the boolean-valued variables $q_r^i \in \{0,1\}$:

\begin{equation} \chi^i = \sum_{r=0}^{R-1} 2^{-r} q_{r}^{i}
\end{equation} 

\begin{equation} x^i = 2\chi^i -1
\end{equation}

From the equation above we can see that the precision of the number heavily depends on the size or R (the number of bits used to represent the number). In the paper by Rogers and Singleton they opt for an R or 4, representing numbers as multiplies of 0.25 in the range of -1 to 3, this leads to a total number of 16 discrete values. 

The new constrained quadratic models (CQM) [2] running on DWaves Hybrid Samplers [3] enable the use of integer values in the quadratic model and therefore drastically expand solution possibilities. Further the possibility of introducing constraints is given. The new constrained quadratic model can be written as:

\begin{equation} H[q] = \sum_{i=1}^{N} a_i x_i + \sum_{i=1}^{N} \sum_{i \neq j}^{N}   b_{ij} x_i x_j  + c
\end{equation} 

- $x_i$: Unknown integer/binary variable we want to optimize for 
- $a_i$: Linear weight (real number)
- $b_{i,j}$: Quadratic term between $i$ and $j$ (real number)
- $c$: Inequality/equality constraints


# Matrix inversion as a Constrained Quadratic model on a DWave Quantum Annealer

To run an optimization on a constrained quadratic model on DWaves Ocean platform we can set the quadratic model containing integers and further add constraints. We import numpy for the classical solution, the dimod ConstrainedQuadraticModel to formulate our problem, the dimod Integer variable, the LeapHybridCQMSampler to sample the solution on the DWave quantum annealer. Ocean does not yet support matrix operations, we import and use Sympy [3] to formulate our problem description.

In [1]:
import numpy as np
from dimod import ConstrainedQuadraticModel, Integer, Real
from dwave.system import LeapHybridCQMSampler
from sympy import sympify, Matrix

### Construct the matrix in sympy
We track our solution variables $x_i$ with the vector components. 

#### 2x2 Test case

In [ ]:
# # 2x2 Test case: successful
M = Matrix([[2, 1], [3, -1]])
X = Matrix([sympify(str('x0')), sympify(str('x1'))])
Y = Matrix([15, 5])
M_C = np.array([[2, 1], [3, -1]])
Y_C = np.array([15, 5])

#### 2x2 Test case: Ill-posed

In [2]:
# # 2x2 Test case: successful
M = Matrix([[1.0, 2.0], [2.0, 3.999]])
X = Matrix([sympify(str('x0')), sympify(str('x1'))])
Y = Matrix([4.0, 7.999])
M_C = np.array([[1.0, 2.0], [2.0, 3.999]])
Y_C = np.array([4.0, 7.999])

#### 3x3 Test case

In [ ]:
M = Matrix([[1, 2, -1], [2, 2, 2], [1, -1, 2]])
X = Matrix([sympify(str('x0')), sympify(str('x1')), sympify(str('x2'))])
Y = Matrix([2, 12, 5])
M_C = np.array([[1, 2, -1], [2, 2, 2], [1, -1, 2]])
Y_C = np.array([2, 12, 5])

#### 3x3 Test case: Ill-posed 

In [ ]:
M = Matrix([[-16.0, 24.0, 4.0], [32.0, -44.0, -8.0], [-12.0, 16.0, 4.0]])
X = Matrix([sympify(str('x0')), sympify(str('x1')), sympify(str('x2'))])
Y = Matrix([3.0, -5.0, 1.0])
M_C = np.array([[-16.0, 24.0, 4.0], [32.0, -44.0, -8.0], [-12.0, 16.0, 4.0]])
Y_C = np.array([3.0, -5.0, 1.0])

#### 4x4 Test case

In [ ]:
M = Matrix([[1, 1 , 1, 1], [2, 3, 0, -1], [-3, 4, 1, 2], [1, 2, -1, 1]])
X = Matrix([sympify(str('x0')), sympify(str('x1')), sympify(str('x2')), sympify(str('x3'))])
Y = Matrix([13, -1, 10, 1])
M_C = np.array([[1, 1 , 1, 1], [2, 3, 0, -1], [-3, 4, 1, 2], [1, 2, -1, 1]])
Y_C = np.array([13, -1, 10, 1])

#### 4x4 Test case: Singular matrix

In [ ]:
M = Matrix([[1, 0 , 1, 0], [0, 1, 0, 1], [1, 1, 0, 0], [0, 0, 1, 1]])
X = Matrix([sympify(str('x0')), sympify(str('x1')), sympify(str('x2')), sympify(str('x3'))])
Y = Matrix([7, 2, 5, 4])
M_C = np.array([[1, 0 , 1, 0], [0, 1, 0, 1], [1, 1, 0, 0], [0, 0, 1, 1]])
Y_C = np.array([7, 2, 5, 4])
# Solution: 3, 2, 4, 0

#### 5x5 Case

In [ ]:
# M = Matrix([[1, 1 , 1, 1, 1], [1, 1 , 1, 1, 1], [1, 1 , 1, 1, 1], [1, 1 , 1, 1, 1], [1, 1 , 1, 1, 1]])
# X = Matrix([sympify(str('x0')), sympify(str('x1')), sympify(str('x2')), sympify(str('x3')), sympify(str('x4'))])
# Y = Matrix([5, 5, 5, 5, 5])
# M_C = np.array([[1, 1 , 1, 1, 1], [1, 1 , 1, 1, 1], [1, 1 , 1, 1, 1], [1, 1 , 1, 1, 1], [1, 1 , 1, 1, 1]])
# Y_C = np.array([5, 5, 5, 5, 5])
# # Solution: 3, 2, 4, 0

### Reformulate the objective functions into components of x as seen above
Following: $H(x) = (Mx-y)^2$

In [3]:
A = M.multiply(X)-Y
result = str(A.T.multiply(A)[0, 0])
print(result)

16.0*(0.25*x0 + 0.5*x1 - 1)**2 + 63.984001*(0.250031253906738*x0 + 0.499937492186523*x1 - 1)**2


In [4]:
# Make it compatible with lists
for i in range(len(X)):
    result = result.replace('x'+str(i), 'x['+str(i)+']')
print(result)

16.0*(0.25*x[0] + 0.5*x[1] - 1)**2 + 63.984001*(0.250031253906738*x[0] + 0.499937492186523*x[1] - 1)**2


#### Paste the above printed reformulation into cqm.set_objective(*printed reformulation*)

In [5]:
#Constraints
lowerBound = -10
upperBound = 10
# Singular matrix case:
# upperBound = 6

#### Create the constrained quadratic model

In [6]:
# Create the quadratic model
cqm = ConstrainedQuadraticModel()
# Create a list of integer variables for each solution component
x = []
for i in range(len(X)):
    x.append(Integer('x'+str(i)))
# Reformulated objective function to minimize here
program = 'cqm.set_objective(' + result + ')'
exec(program)
# # Add constraints, here non-negativity and range constraint
for c in range(len(X)):
    cqm.add_constraint(x[c] >= lowerBound, label='constraint_' + str(2*c))
    cqm.add_constraint(x[c] <= upperBound, label='constraint_' + str(2*c+1))
# Substitute integers with self loops
cqm.substitute_self_loops()

ValueError: REAL variables (e.g. 'x0') cannot have interactions

#### Run the constrained quadratic model on the hybrid sampler

In [ ]:
# Sample from the Hybrid Solver on our constrained quadratic model
sampleset = LeapHybridCQMSampler().sample_cqm(cqm, label="CQM Matrix Inversion")
# Filter feasible solutions
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible) 
# Print...
# print(sampleset)
print(feasible_sampleset.first) 

### Classical solution
Solution using the classical inverse approximation.

In [ ]:
print("Classical solution: ", np.linalg.inv(M_C).dot(Y_C))
print("Condition number: ", np.linalg.cond(M_C))

### References 
- [1] Rogers, Michael L., and Robert L. Singleton Jr. "Floating-point calculations on a quantum annealer: Division and matrix inversion." Frontiers in Physics 8 (2020): 265.
- [2] https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/constrained.html
- [3] https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html
- [4] https://docs.sympy.org/latest/index.html